In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline
pd.options.display.max_columns=999

In [2]:
path = 'Files/' 
età = pd.read_csv(path+'distribuzione_regionale_età.csv')
letalità = pd.read_csv(path+'letalità_Covid19.csv')
posti_ti = pd.read_csv(path+'Posti_letto_terapia_intensiva.csv')
regioni = pd.read_json('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-json/dpc-covid19-ita-regioni.json')

In [3]:
for index,row in letalità.iterrows():
    età.loc[(età['Età']>= row['min'])&(età['Età']<= row['max']),'fascia_età'] = row['fascia_età']
    età.loc[(età['Età']>= row['min'])&(età['Età']<= row['max']),'letalità'] = row['letalità']
    
età = età.groupby(['Territorio','fascia_età','letalità'], as_index = False)['Valore'].sum()
età['percentuale_età'] = età.groupby(['Territorio'])['Valore'].apply(lambda x: x / float(x.sum()))

In [4]:
regione_casi = età.merge(regioni, left_on='Territorio', right_on='denominazione_regione', how= 'left')
regione_casi = regione_casi.merge(posti_ti, on='Territorio', how='left')
# calcoliamo la saturazione dei posti letto
regione_casi['TI_saturazione'] = regione_casi['terapia_intensiva'] / regione_casi['Posti letto TI']
# filtriamo le colonne che ci interessano
regione_casi = regione_casi[['Territorio','data','fascia_età','letalità','percentuale_età','TI_saturazione','totale_casi','deceduti','tamponi']]

In [5]:
regione_casi.loc[regione_casi['Territorio']== 'Emilia Romagna'].head(10)

,Territorio,data,fascia_età,letalità,percentuale_età,TI_saturazione,totale_casi,deceduti,tamponi
1044,Emilia Romagna,2020-02-24 18:00:00,10-19,0.002,0.090798,0.004608,18,0,148
1045,Emilia Romagna,2020-02-25 18:00:00,10-19,0.002,0.090798,0.004608,26,0,391
1046,Emilia Romagna,2020-02-26 18:00:00,10-19,0.002,0.090798,0.006912,47,1,577
1047,Emilia Romagna,2020-02-27 18:00:00,10-19,0.002,0.090798,0.013825,97,1,1033
1048,Emilia Romagna,2020-02-28 18:00:00,10-19,0.002,0.090798,0.013825,145,2,1277
1049,Emilia Romagna,2020-02-29 17:00:00,10-19,0.002,0.090798,0.025346,217,4,1550
1050,Emilia Romagna,2020-03-01 17:00:00,10-19,0.002,0.090798,0.029954,285,8,1795
1051,Emilia Romagna,2020-03-02 18:00:00,10-19,0.002,0.090798,0.036866,335,11,1973
1052,Emilia Romagna,2020-03-03 18:00:00,10-19,0.002,0.090798,0.055300,420,18,2012
1053,Emilia Romagna,2020-03-04 17:00:00,10-19,0.002,0.090798,0.059908,544,22,2500
